# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [110]:
!pip install ultralytics

## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [111]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [112]:
from ultralytics import YOLO
model = YOLO('/content/drive/MyDrive/Yolo T5/Dataset/YOLO_Model.pt')


## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [113]:
import cv2
video_path = '/content/drive/MyDrive/Yolo T5/Dataset/YOLO_Video.mp4'
video = cv2.VideoCapture(video_path)

## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [114]:
Width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
Height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
Fps = int(video.get(cv2.CAP_PROP_FPS))

In [115]:
ret, frame = video.read()
results = model(frame)


0: 384x640 2 buss, 90 cars, 2 trucks, 2 vans, 63.7ms
Speed: 2.4ms preprocess, 63.7ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)


## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [116]:
outPut_video='/content/outPut_video.mp4'
FourCC =cv2.VideoWriter_fourcc(*'mp4v')
outPut=cv2.VideoWriter(outPut_video , FourCC ,Fps ,(Width, Height))

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

In [117]:
TOP = [(450, 360), (600, 550)]
BOTTOM = [(1100, 900), (1250, 700)]
RIGHT = [(750, 850), (800, 950)]
LEFT = [(1000, 600), (1250, 300)]

## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

In [118]:
coloe1=(255,204,0)
coloe2=(153,153,255)
coloe3=(255,255,204)
coloe4=(204,255,255)

In [122]:
import cv2
import numpy as np

TOP_cont = 0
BOTTOM_cont = 0
RIGHT_cont = 0
LEFT_cont = 0

while video.isOpened():
    ret, frame = video.read()
    if not ret:
        break

    results = model.predict(frame)

    for detection in results[0]:
        class_id = detection.boxes.cls.cpu().numpy().astype(int)
        confidence = detection.boxes.conf.cpu().numpy()
        xyxy = detection.boxes.xyxy.cpu().numpy()

        for i in range(len(class_id)):
            x1, y1, x2, y2 = map(int, xyxy[i])
            x = x1
            y = y1
            w = x2 - x1
            h = y2 - y1
            X_center = int(x + w / 2)
            Y_center = int(y + h / 2)

            cv2.rectangle(frame, (x, y), (x2, y2), coloe2, 2)
            cv2.circle(frame, (X_center, Y_center), 5, coloe2, -1)
            cv2.putText(frame, f'ID: {i} Class: {class_id[i]} Conf: {confidence[i]:.2f}',
                (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, coloe2, 2)


    cv2.rectangle(frame, TOP[0], TOP[1], coloe3, 2)
    cv2.rectangle(frame, BOTTOM[0], BOTTOM[1], coloe3, 2)
    cv2.rectangle(frame, RIGHT[0], RIGHT[1], coloe3, 2)
    cv2.rectangle(frame, LEFT[0], LEFT[1], coloe3, 2)

    cv2.putText(frame, 'Ahdab Albishri',(10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, coloe1, 2)
    cv2.putText(frame, f"Top: {TOP_cont}", (15, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, coloe4, 2)
    cv2.putText(frame, f"Bottom: {BOTTOM_cont}", (15, 100), cv2.FONT_HERSHEY_SIMPLEX, 1,coloe4, 2)
    cv2.putText(frame, f"Left: {LEFT_cont}", (15, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, coloe4, 2)
    cv2.putText(frame, f"Right: {RIGHT_cont}", (15, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, coloe4, 2)

    outPut.write(frame)

video.release()
outPut.release()
print(f"Video saved to {outPut_video}")


0: 384x640 3 buss, 103 cars, 1 truck, 4 vans, 62.9ms
Speed: 3.2ms preprocess, 62.9ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 buss, 104 cars, 1 truck, 5 vans, 60.2ms
Speed: 3.1ms preprocess, 60.2ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 buss, 103 cars, 1 truck, 5 vans, 47.0ms
Speed: 3.1ms preprocess, 47.0ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 buss, 104 cars, 1 truck, 5 vans, 47.0ms
Speed: 2.5ms preprocess, 47.0ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 buss, 102 cars, 1 truck, 4 vans, 47.0ms
Speed: 3.0ms preprocess, 47.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 buss, 102 cars, 1 truck, 3 vans, 44.5ms
Speed: 3.2ms preprocess, 44.5ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 buss, 101 cars, 1 truck, 3 vans, 44.4ms
Speed: 3.2ms preprocess, 44.4ms inferenc

## Save and Submit Your Work

Submit both the notebook and the output video